In [1]:
import os
import re
import requests
import time
import pandas as pd
import numpy as np
import torch 
from torch import nn
from torch.optim import AdamW  
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd 
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer, pipeline, BertTokenizer, BertModel
from transformers import Trainer, TrainingArguments
import spacy
from spacy import displacy

from sklearn.model_selection import train_test_split
from tqdm import tqdm
torch.set_default_device("cpu")
import random
import json

In [11]:
pwd

'/Users/chienshiyun/Documents/_DSA3101/proj/dsa3101'

In [13]:
import re
dfs = {}
pdf_folder_path = "datasets_in_json/"

for industry in os.listdir(pdf_folder_path):
    industry_path = os.path.join(pdf_folder_path, industry)  # Corrected variable name
    if os.path.isdir(industry_path):  # Check if it's a directory for industry
        for country in os.listdir(industry_path):  # Loop through countries in the industry folder
            country_path = os.path.join(industry_path, country)
            if os.path.isdir(country_path):  # Check if it's a directory for country
                json_files = [f for f in os.listdir(country_path) if f.endswith('.json')]
                for file in json_files:
                    match = re.match(r"(.+?)_(\d{4})_ocr\.json", file)
                    file_path = os.path.join(country_path, file)
                    print(f"Processing file: {file_path}")

                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if isinstance(data, list):  
                        df = pd.DataFrame(data, columns=["esg_text"])
                        df["country"] = country
                        df["industry"] = industry
                        if match:
                            company_name, year = match.groups()
                            df["company"] = company_name
                            df["year"] = int(year)
                        dfs[file] = df  # Store DataFrame with file name as key
                    else:
                        print(f"Skipping {file}: Unsupported format")
                        continue


Processing file: datasets_in_json/Energy/Australia/Origin_2024_ocr.json
Processing file: datasets_in_json/Energy/Australia/Genex_2023_ocr.json
Processing file: datasets_in_json/Energy/SA/Energisa_2023 _ocr.json
Processing file: datasets_in_json/Energy/SA/Soltec_2023_ocr.json
Processing file: datasets_in_json/Energy/Europe/Enel_2023_ocr.json
Processing file: datasets_in_json/Health/NA/Pfizer_2022_ocr.json
Processing file: datasets_in_json/Industrial/NA/JohnsonControl_2024_ocr.json
Processing file: datasets_in_json/Industrial/NA/3M_2023_ocr.json
Processing file: datasets_in_json/Industrial/NA/SPX_Flow_2024_ocr.json
Processing file: datasets_in_json/Industrial/NA/Nordson_2023_ocr.json
Processing file: datasets_in_json/Industrial/NA/MPMaterials_2024_ocr.json
Processing file: datasets_in_json/Industrial/Australia/RioTinto_2022_ocr.json
Processing file: datasets_in_json/Industrial/Asia/Hanwha_Solutions_2024_ocr.json
Processing file: datasets_in_json/Industrial/Asia/Mencast_2021_ocr.json
Proc

In [18]:
dfs['Origin_2024_ocr.json']

,esg_text,country,industry,company,year
0,Contents 000 0 000 0 Origin’s 2024 Sustainabil...,Australia,Energy,Origin,2024
1,See our GRI content index and Our Sustainabili...,Australia,Energy,Origin,2024
2,"corporate reporting suite, which includes: sus...",Australia,Energy,Origin,2024
3,© 2024 Annual Report A message from the CEO 3 ...,Australia,Energy,Origin,2024
4,wirttteeessescess eee cere cere cree cece eee ...,Australia,Energy,Origin,2024
...,...,...,...,...,...
913,"Where data is not available due to timing, we ...",Australia,Energy,Origin,2024
914,"Where applicable, we revise prior year data in...",Australia,Energy,Origin,2024
915,Additional emissions information is available ...,Australia,Energy,Origin,2024
916,1.,Australia,Energy,Origin,2024


# Data Cleaning
1. Remove duplicate data 
2. Handle missing values

In [19]:
df_combined = pd.concat(dfs.values(), ignore_index=True)
missing_rows = df_combined[df_combined.isnull().any(axis=1)]
print(len(missing_rows))
df_combined.dropna(inplace=True)
df_combined.drop_duplicates(inplace=True)

1091


In [22]:
df_combined["esg_text"].apply(type).value_counts()

esg_text
<class 'str'>    63903
Name: count, dtype: int64

In [23]:
df_combined.to_csv('combined_files/not_categorised.csv', index=False)

# Data Transformation

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
keywords = [
    "Environment", "Energy", "Waste", "Pollutants", "Water", "Climate",
    "Decarbonisation", "Carbon", "emissions", "Strategy", "Social", "Labor", "Practices", "Human Rights",
    "Human Capital", "Capital", "Management", "Occupational", "Health" ,"Safety", "Financial Inclusion",
    "Community investment", "Customer","Relations", "Privacy", "Protection",
    "Gender", "Women", "Female", "Race", "Ethnic", "Diversity", "Governance", "Transparency", "Reporting", "Corporate Governance",
    "Materiality", "Risk", "Crisis",  "Management", "Ethics", "Policy",
    "Tax Strategy", "Shareholder", "Information Security", "Cybersecurity", "System Availability","Sustainable", "Sustainable Finance", "Board Diversity"
]
keywords_regex = "|".join([f"\\b{k}\\b" for k in keywords])  # \b ensures whole-word matching


def extract_ner(text):
    if pd.isna(text):
        return None
    
    doc = nlp(text)
    entities = {"numbers": [], "metrics": [], "dates": []}

    for ent in doc.ents:
        if ent.label_ in ["CARDINAL", "ORDINAL"]:  # Numbers (e.g., "10", "zero", "first")
            entities["numbers"].append(ent.text)
        elif ent.label_ in ["QUANTITY", "PERCENT", "MONEY"]:  # Metrics (e.g., "5kg", "30%", "$100")
            entities["metrics"].append(ent.text)
        elif ent.label_ in ["DATE", "TIME"]:  # Dates/Time (e.g., "January 1, 2024", "3 PM")
            entities["dates"].append(ent.text)

    return entities if any(entities.values()) else None


df_combined["NER_Results"] = df_combined["esg_text"].apply(extract_ner)
df_filtered = df_combined[
    df_combined["NER_Results"].notna() |  #df_combined.dropna(subset=["NER_Results"])
    df_combined["esg_text"].str.contains(keywords_regex, case=False, na=False, regex=True)
]


In [ ]:
print(len(df_combined))
print(len(df_filtered))

In [ ]:
df_filtered.to_csv('../files/combined_pdfs_filteredNER_2802.csv', index=False)

In [ ]:
df_filtered.head(5)

# Classification

In [30]:
from transformers import BertTokenizer, BertForSequenceClassification
MODEL_NAME = "nbroad/ESG-BERT"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class ESGClassifier(nn.Module):
    def __init__(self, n_classes=28):
        super(ESGClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME) 
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.classifier(output)

In [32]:
MODEL_PATH = "esgbert_model_full.pth"
model = ESGClassifier()
model = torch.load(MODEL_PATH, map_location=device)  
model.eval()

ESGClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [39]:
type(df_combined["esg_text"])

pandas.core.series.Series

In [42]:
text = df_combined["esg_text"].head(5).tolist()
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
if 'token_type_ids' in inputs:
    del inputs['token_type_ids']
with torch.no_grad():
    outputs = model(**inputs)

# If the model returns a dictionary, access `logits`
if hasattr(outputs, "logits"):  
    logits = outputs.logits
else:
    logits = outputs  # Assume it's already a tensor

# Apply softmax to logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)

print(type(outputs))  # Check whether it's a Tensor or a dictionary
print(outputs)        

<class 'torch.Tensor'>
tensor([[ 3.2543e+00,  3.5519e+00,  1.9438e+00, -6.3103e-01,  1.6889e+00,
          1.6461e+00,  2.2837e+00, -3.0058e+00, -3.7740e+00, -4.0991e+00,
         -3.7867e+00, -2.6847e+00, -1.2956e+00, -1.1589e+00,  6.7044e-01,
         -3.0336e+00, -3.9898e+00, -2.6495e+00,  1.8689e+00, -3.1223e+00,
         -1.4244e+00, -2.2749e+00, -2.7990e+00, -3.9395e-01, -2.3462e+00,
         -1.0446e+00,  4.3820e+00, -2.2058e+00],
        [ 3.4920e+00, -4.1984e-01,  1.2028e+00, -1.0238e+00,  1.9961e+00,
          1.4623e+00,  3.0394e+00, -4.4927e+00, -4.4525e+00, -5.5914e+00,
         -4.4648e+00, -2.4135e+00, -1.9246e+00, -8.3860e-01, -1.5488e+00,
         -3.3229e+00, -3.1849e+00, -2.8936e+00,  3.8835e+00, -1.7895e+00,
          8.2715e-01, -4.8941e-02, -4.0159e+00, -7.6370e-01, -3.4307e+00,
         -2.1421e+00,  4.5566e+00, -2.9161e+00],
        [ 3.6965e+00, -1.4975e+00, -2.4553e-01, -1.8256e+00,  2.3617e+00,
          1.3081e+00,  1.6035e+00, -4.5593e+00, -3.2147e+00, -4.8

In [ ]:
candidate_labels = ["No Label",
                    "Environment - Energy efficiency", 
                    "Environment - Waste & Pollutants Generation", 
                    "Environment - Water Usage", 
                    "Environment - Climate Strategy", 
                    "Environment - Decarbonisation/Carbon emissions", 
                    "Environment - Strategy",
                    "Social - Gender and Ethnic Diversity",
                    "Social - Labor Practices",
                    "Social - Human Rights", 
                    "Social - Human Capital Management",
                    "Social - Occupational Health & Safety",
                    "Social - Financial Inclusion",           
                    "Social - Customer Relations",
                    "Social - Privacy Protection",
                    "Social - Community investment",
                    "Social - Human Capital Management",
                    "Governance - Board Diversity",
                    "Governance - Transparency & Reporting", 
                    "Corporate Governance", 
                    "Governance - Materiality", 
                    "Governance - Risk & Crisis Management",              
                    "Governance - Business Ethics", 
                    "Governance - Policy Influence", 
                    "Governance - Tax Strategy", 
                    "Governance - Information Security/ Cybersecurity & System Availability", 
                    "Governance - Sustainable Finance",
                    "Governance - Shareholder rights"]

from tqdm import tqdm
tqdm.pandas()

THRESHOLD=0.5
def classify_text(text):
    if pd.isna(text): 
        return None

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
        
    with torch.no_grad():
        outputs = model(**inputs)
        
    if hasattr(outputs, "logits"):  
        logits = outputs.logits
    else:
        logits = outputs  # Assume it's already a tensor


    probabilities = torch.nn.functional.softmax(logits, dim=-1)  # Softmax for classification
    probabilities = probabilities.squeeze().cpu().numpy()

    if len(probabilities) != len(candidate_labels):
        print(f"Warning: Mismatch! Probabilities: {len(probabilities)}, Labels: {len(candidate_labels)}")
        return ["Error"]

    assigned_labels = [candidate_labels[i] for i, prob in enumerate(probabilities) if prob > THRESHOLD]
    return assigned_labels if assigned_labels else ["No Label"]

tqdm.pandas()


df_combined["labels"] = df_combined["esg_text"].progress_apply(classify_text)

  0%|                                                              | 0/63903 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 29%|█████████████▋                                  | 18248/63903 [44:01<8:13:21,  1.54it/s]

In [ ]:
df_combined.head(5)

In [ ]:
df_combined['labels'].value_counts()

In [ ]:
df_combined.to_csv('files/labeled_pdfs_1003.csv', index=False)

# Add into Table

In [ ]:
# import psycopg2
# from datetime import datetime

# # PostgreSQL connection
# conn = psycopg2.connect(
#     dbname="your_database",
#     user="your_user",
#     password="your_password",
#     host="localhost",
#     port="5432"
# )
# cursor = conn.cursor()

# # Example extracted ESG data
# esg_data = {
#     "company_name": "Pfizer",
#     "year": 2023,
#     "carbon_emissions_reduction": 7.5,
#     "renewable_energy_usage": 8.0,
#     "waste_management": 7.0,
#     "workforce_diversity": 6.5,
#     "employee_rights": 8.0,
#     "product_safety": 7.5,
#     "board_independence": 8.5,
#     "transparency": 8.0,
#     "executive_pay_equity": 6.0,
#     "anti_corruption_policies": 7.5,
#     "overall_esg_score": 7.43
# }

# # SQL INSERT query
# query = """
# INSERT INTO ESG_Scores (company_name, year, carbon_emissions_reduction, renewable_energy_usage,
#                         waste_management, workforce_diversity, employee_rights, product_safety,
#                         board_independence, transparency, executive_pay_equity, anti_corruption_policies, 
#                         overall_esg_score, created_at) 
# VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# """

# # Execute query
# cursor.execute(query, (
#     esg_data["company_name"], esg_data["year"], esg_data["carbon_emissions_reduction"],
#     esg_data["renewable_energy_usage"], esg_data["waste_management"], esg_data["workforce_diversity"],
#     esg_data["employee_rights"], esg_data["product_safety"], esg_data["board_independence"],
#     esg_data["transparency"], esg_data["executive_pay_equity"], esg_data["anti_corruption_policies"],
#     esg_data["overall_esg_score"], datetime.now()
# ))

# # Commit and close
# conn.commit()
# cursor.close()
# conn.close()

# print("ESG data inserted successfully!")
